In [45]:
!pip install -U spacy

In [3]:
# Português
!python -m spacy download pt_core_news_sm

[+] Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')


In [3]:
# Carregue o modelo
import spacy
import pt_core_news_sm

nlp = pt_core_news_sm.load()

In [4]:
texto = nlp('Maria mora em Paris')

# Interando sobre as entidades identificadas na frase
for entidade in texto.ents:
  print(entidade.text, entidade.label_)

Maria PER
Paris LOC


In [5]:
# Destacando visualmente as entidades da frase e seus tipos
spacy.displacy.render(texto, style='ent', jupyter=True)

In [6]:
# Visualização do Analisador de Dependência
spacy.displacy.render(texto, style='dep',jupyter=True)

In [7]:
# Ver a definição do tipo de entidade
print("PER:", spacy.explain('PER'))
print("LOC:", spacy.explain('LOC'))
print("ORG:", spacy.explain('ORG'))
print("MISC:", spacy.explain('MISC'))

PER: Named person or family.
LOC: Non-GPE locations, mountain ranges, bodies of water
ORG: Companies, agencies, institutions, etc.
MISC: Miscellaneous entities, e.g. events, nationalities, products or works of art


In [8]:
text = nlp("Juliana nasceu no dia 18/06/2018 em Fortaleza")
spacy.displacy.render(text, style='ent', jupyter=True)

In [9]:
for entidade in text.ents:
  print(entidade.text, entidade.label_)

Juliana PER
Fortaleza LOC


In [10]:
import random

In [34]:
TRAIN_DATA = [
    ("Almira é uma ótima pessoa, gosto muito dela", {"entities": [(0, 6, "PER")]}),
    ("João foi para Bahia nas férias", {"entities": [(14, 19, "LOC"), (0, 4, "PER")]}),
    ("Carlos foi visitar João na casa de praia", {"entities": [(0, 6, "PER"), (19, 23, "PER")]}),
    ("No meio do ano irei para São Paulo fazer mais um curso", {"entities": [(25, 34, "LOC")]}),
    ("O sonho dela era ir para Austrália visitar seu irmão", {"entities": [(25, 34, "LOC")]}),
    ("Em 15/07/1988 nasceu essa linda criança", {"entities": [(3, 13, "DATE"), (18, 24, "LOC")]}),
    ("Data de prisão: 10/01/2018", {"entities": [(16, 26, "DATE")]}),
    ("No dia 01/02/2016 foi decretada a sentença", {"entities": [(7, 17, "DATE")]}),
    ("A data da festa foi 07/05/2018", {"entities": [(20, 30, "DATE")]}),
    ("Dia 07/06/2020 choveu pela manhã", {"entities": [(4, 14, "DATE")]}),
    ("Michael Jackson nasceu dia 29/08/1958", {"entities": [(0, 15, "PER"), ( 27, 37, "DATE")]}),
    ("A viagem foi marcada para o dia 21/03/2021", {"entities": [(32, 42, "DATE")]}),
    ("O casamento, que era dia 12/06/2020, foi adiado para o dia 20/07/2020", {"entities": [(25, 35, "DATE"), (59, 68, "DATE")]}),
    ("Dia 21/04/2020 foi feriado", {"entities": [(4, 14, "DATE")]}),
    ("A entrega foi realizada na manhã do dia 18/03/2020 ", {"entities": [(40, 50, "DATE")]}),
    ("Na noite do dia 17/05/2019 aconteceu um eclipse", {"entities": [(16, 26, "DATE")]}),
    ("Fortaleza é a capital do Ceará", {"entities": [(0, 9, "LOC"), (25, 30, "LOC")]})
]

In [39]:
def train_spacy(data, n_iter):
  TRAIN_DATA = data
  """Carrega o modelo, configura o pipeline e treina o reconhecedor da entidade."""
  nlp = pt_core_news_sm.load()  # Carrega um modelo do spaCy existente
  print("Modelo carregado")

  # Criando os componentes internos do pipeline e adicionando-os ao pipeline
  # nlp.create_pipe funciona para built-ins registrados com spaCy
  if "ner" not in nlp.pipe_names:
    ner = nlp.create_pipe("ner")
    nlp.add_pipe(ner, last=True)
  # Caso contrário, obtenha-o para que possa adicionar as labels
  else:
    ner = nlp.get_pipe("ner")

  # Adicionando as labels
  for _, annotations in TRAIN_DATA:
    for ent in annotations.get('entities'):
      ner.add_label(ent[2])

  # Obtendo os nomes de outros pipes para desativá-los durante o treinamento
  other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
  with nlp.disable_pipes(*other_pipes):  # Treina somente NER
    spacy.util.fix_random_seed()
    optimizer = nlp.create_optimizer()
    
    for itn in range(n_iter):
      random.seed(10)
      random.shuffle(TRAIN_DATA)
      losses = {}
      
      for texts, annotations in TRAIN_DATA:
        nlp.update(
            [texts],  # batch de textos
            [annotations],  # batch de anotações
            drop=0.2,  # dropout - dificulta a memorização de dados
            sgd=optimizer,
            losses=losses,
        )
      print("Losses", losses)

   # test the trained model
    for text, _ in TRAIN_DATA:
      doc = nlp(text)
      print("Entities", [(ent.text, ent.label_) for ent in doc.ents])
      print("Tokens", [(t.text, t.ent_type_) for t in doc])
      print("\n")
    return nlp

In [40]:
nlp_ = train_spacy(data=TRAIN_DATA, n_iter=100)

Modelo carregado


ValueError: [E989] `nlp.update()` was called with two positional arguments. This may be due to a backwards-incompatible change to the format of the training data in spaCy 3.0 onwards. The 'update' function should now be called with a batch of Example objects, instead of `(text, annotation)` tuples. 